# 初学者的 TensorFlow 2.0 教程

In [1]:
import tensorflow as tf

如果您在自己的开发环境而不是 [Colab](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/quickstart/beginner.ipynb) 中操作，请参阅设置 TensorFlow 以进行开发的[安装指南](https://tensorflow.google.cn/install)。

注：如果您使用自己的开发环境，请确保您已升级到最新的 `pip` 以安装 TensorFlow 2 软件包。有关详情，请参阅[安装指南](https://tensorflow.google.cn/install)。

## 加载数据集

加载并准备 [MNIST 数据集](http://yann.lecun.com/exdb/mnist/)。将样本数据从整数转换为浮点数：

In [2]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

11490434/11490434 [==============================] - 1s 0us/step


## 构建机器学习模型

通过堆叠层来构建 `tf.keras.Sequential` 模型。

In [3]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

对于每个样本，模型都会返回一个包含 [logits](https://developers.google.com/machine-learning/glossary#logits) 或 [log-odds](https://developers.google.com/machine-learning/glossary#log-odds) 分数的向量，每个类一个。

In [4]:
predictions = model(x_train[:1]).numpy()
predictions

array([[ 0.08706942, -0.24765234, -0.35373405,  0.13567466, -0.15034369,
        -0.00750189, -0.76972854, -0.00473149, -0.6176619 , -0.04062864]],
      dtype=float32)

`tf.nn.softmax` 函数将这些 logits 转换为每个类的*概率*： 

In [5]:
tf.nn.softmax(predictions).numpy()

array([[0.12790248, 0.09151897, 0.0823077 , 0.13427277, 0.10087226,
        0.11636093, 0.05429706, 0.11668374, 0.0632147 , 0.11256941]],
      dtype=float32)

注：可以将 `tf.nn.softmax` 烘焙到网络最后一层的激活函数中。虽然这可以使模型输出更易解释，但不建议使用这种方式，因为在使用 softmax 输出时不可能为所有模型提供精确且数值稳定的损失计算。 

使用 `losses.SparseCategoricalCrossentropy` 为训练定义损失函数，它会接受 logits 向量和 `True` 索引，并为每个样本返回一个标量损失。

In [6]:
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

此损失等于 true 类的负对数概率：如果模型确定类正确，则损失为零。

这个未经训练的模型给出的概率接近随机（每个类为 1/10），因此初始损失应该接近 `-tf.math.log(1/10) ~= 2.3`。

In [7]:
loss_fn(y_train[:1], predictions).numpy()

2.1510584

在开始训练之前，使用 Keras `Model.compile` 配置和编译模型。将 [`optimizer`](https://tensorflow.google.cn/api_docs/python/tf/keras/optimizers) 类设置为 `adam`，将 `loss` 设置为您之前定义的 `loss_fn` 函数，并通过将 `metrics` 参数设置为 `accuracy` 来指定要为模型评估的指标。

In [8]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

## 训练并评估模型

使用 `Model.fit` 方法调整您的模型参数并最小化损失： 

In [9]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3033 - accuracy: 0.9121
Epoch 2/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1483 - accuracy: 0.9559
Epoch 3/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1136 - accuracy: 0.9659
Epoch 4/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0920 - accuracy: 0.9712
Epoch 5/5
1875/1875 [==============================] - 4s 2ms/step - loss: 0.0778 - accuracy: 0.9758


`Model.evaluate` 方法通常在 "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" 或 "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)" 上检查模型性能。

In [10]:
model.evaluate(x_test,  y_test, verbose=2)

313/313 - 1s - loss: 0.0678 - accuracy: 0.9793 - 652ms/epoch - 2ms/step


[0.06776973605155945, 0.9793000221252441]

现在，这个照片分类器的准确度已经达到 98%。想要了解更多，请阅读 [TensorFlow 教程](https://tensorflow.google.cn/tutorials/)。

如果您想让模型返回概率，可以封装经过训练的模型，并将 softmax 附加到该模型：

In [11]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [12]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[1.1428954e-07, 1.0422523e-08, 5.9129384e-06, 9.8103570e-05,
        9.6591572e-12, 1.3992313e-07, 5.2475450e-13, 9.9989188e-01,
        5.8532737e-07, 3.2857463e-06],
       [4.9927507e-08, 2.3798124e-05, 9.9997592e-01, 1.4980915e-07,
        1.0107405e-16, 3.9909061e-08, 1.7710319e-08, 5.7444548e-14,
        1.0519833e-07, 6.2000047e-14],
       [1.9570288e-07, 9.9805766e-01, 3.6780885e-04, 5.8330579e-06,
        5.7589677e-06, 3.6658951e-06, 1.9785537e-05, 1.3881406e-03,
        1.5054835e-04, 5.4826592e-07],
       [9.9997282e-01, 4.9455590e-10, 1.6936481e-05, 2.1083267e-08,
        6.0458056e-09, 1.3322247e-06, 2.4969354e-06, 6.1352798e-06,
        6.0684728e-09, 2.9052225e-07],
       [3.6599366e-05, 4.6704272e-09, 1.4177052e-05, 8.5130800e-09,
        9.9715364e-01, 2.1816984e-06, 2.1543970e-05, 3.1760120e-04,
        5.0043982e-06, 2.4493090e-03]], dtype=float32)>